# Chap06 - 텍스트와 시퀀스를 위한 딥러닝

## Goals

- 텍스트 데이터를 유용한 형태로 전처리하는 방법
- 순환 신경망(RNN)을 사용하는 방법
- 1D 컨브넷을 사용한 시퀀스 데이터의 처리

## Intro

- 시퀀스(sequence) 데이터를 처리하는 기본적인 딥러닝 모델을 **순환 신경망(RNN, Recurrent Neural Network)**과 **1D 컨브넷(1D convnet)** 두 가지다.
  - 1D 컨브넷은 2D 컨브넷의 1차원 버전이다.
- 다음과 같은 예제에 사용할 수 있다.
  - 문서 분류나 시계열 분류. → 글의 주제나 책의 저자 식별하기
  - 시계열 비교. → 두 문서나 두 주식 가격이 얼마나 밀접하게 관련이 있는지 추정하기
  - 시퀀스-투-시퀀스 학습. → 영어 문장을 한국어로 번역
  - 감성 분석 → 영화 리뷰 긍정/부정 분류하기
  - 시계열 예측 → 향후 날씨 예측하기

## 6.1 텍스트 데이터 다루기

- 텍스트는 가장 흔한 시퀀스 형태의 데이터다.
  - 텍스트는 단어의 시퀀스나 문자의 시퀀스로 이해할 수 있다.
  - 보통 단어 수준으로 작업하는 경우가 많다.

- 시퀀스 처리용 딥러닝 모델은 텍스트를 사용하여 기초적인 자연어 이해(NLU, natural language understanding) 문제를 처리할 수 있다.
- 이러한 모델은 문자 언어(written language)에 대한 **통계적 구조**를 만들어 간단한 텍스트 문제를 해결한다.
- 자연어 처리(NLP, natural language processing)를 위한 딥러닝은 **단어, 문장, 문단**에 적용한 패턴인식이다.
- 딥러닝 모델의 입력으로 텍스트 원본을 사용하지 못하기 때문에 **텍스트 → 텐서** 로 변환해줘야 한다 → **텍스트 벡터화(vectorizing text)**
- 텍스트 벡터화에는 여러가지 방식이 있다.
  - 텍스트를 **단어(word)**로 나누고 각 단어를 하나의 벡터로 변환한다.
  - 텍스트를 **문자(character)** 로 나누고 각 문자를 하나의 벡터로 변환한다.
  - 텍스트에서 단어나 문자의 **n-gram**을 추출하여 각 n-gram을 하나의 벡터로 변환한다.
- 텍스트를 나누는 단위(단어, 문자, n-gram)을 **토큰(token)**이라고 하며, 토큰으로 나누는 작업을 **토큰화(tokenization)** 라고 한다.
- 모든 텍스트 벡터화 과정은 어떤 종류의 토큰화를 적용하고 **생성된 토큰에 벡터를 연결하는 것** 으로 이루어진다.
  - 이 벡터는 시퀀스 텐서로 묶여져서 신경망에 주입된다.
- 토큰과 벡터를 연결하는 방법은 여러가지가 있는데, 대표적인 두 가지는 다음과 같다.
  - **원-핫 인코딩(one-hot encoding)**
  - **토큰 임베딩(token embedding)** = **단어 임베딩(word embedding)**

### 6.1.1 단어와 문자의 원-핫 인코딩

원-핫 인코딩은 토큰을 벡터로 변환하는 가장 일반적이고 기본적인 방법입니다. [3장에서 IMDB](https://github.com/ExcelsiorCJH/Deep-Learning-with-Python/blob/master/Chap03-Getting_started_with_neural_networks/3.4-classifying_movie_reviews.ipynb)와 [로이터 예제](https://github.com/ExcelsiorCJH/Deep-Learning-with-Python/blob/master/Chap03-Getting_started_with_neural_networks/3.5-classifying_newswires.ipynb)에서 이를 보았습니다(단어의 원-핫 인코딩을 사용했습니다). 

모든 단어에 **고유한 정수 인덱스**를 부여하고 이 정수 인덱스 i를 크기가 N(어휘 사전의 크기)인 이진 벡터로 변환합니다. 이 벡터는 **i번째 원소만 1이고 나머지는 모두 0**입니다.

물론 원-핫 인코딩은 문자 수준에서도 적용할 수 있습니다. 원-핫 인코딩이 무엇이고 어떻게 구현하는지 명확하게 설명하기 위해 단어와 문자에 대한 간단한 예를 만들었습니다.

단어 수준의 원-핫 인코딩(간단한 예):

In [1]:
import numpy as np

# 초기 데이터: 각 원소가 샘플입니다
# (이 예에서 하나의 샘플이 하나의 문장입니다. 하지만 문서 전체가 될 수도 있습니다)
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 데이터에 있는 모든 토큰의 인덱스를 구축한다.
